# **Week 1 : Linear Algebra and Calculus**

In this assignment, we shall explore the concepts of analytic and numeric computation of gradients. Further, we will have a look at the computation graph which is a central concept to building a neural network. For learning how gradients are computed analytically, refer to the resources provided in this week.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQWsYeD8ZlYxFBB33qIn7bwQvP-KuvLZXJOoA&usqp=CAU"
 style="float:center;width:50px;height:50px;">

# **Importing Libraries**
Feel free to import any additional libraries required

In [174]:
# Import all libraries here
import numpy as np


# Setting the seed for reproducible results


# *Problem 1(a)*

In this problem, we shall be exploring the concepts of analytic and numeric computation of gradients for scalar valued functions. 

\begin{equation}
z = w^{T}x + b \\ 
\hat{y} = \sigma(z) = \frac{1}{1+e^{-z}}\\ 
L(\hat{y}, y) = y.log(\hat{y}) + (1-y).log(1-\hat{y})
\end{equation}

For this set of equations, the vector w, and scalars b, y are to be treated as constants. \\

Now, let us find the analytic gradient of the function L with respect to the function x. That is, compute $\frac{\delta L}{\delta x}$. Write the answer obtained as code in the function provided.

Initialise $w$ to a $10 \times 1$ vector sampled over a standard multivariate gaussian distribution, b to a uniform random variable over the interval $(0, 1)$, y to a bernoulli random variable over $\{0, 1\}$

In [175]:
# Initialisation : 

w = np.random.multivariate_normal([0],[[1]],10)
b = np.random.uniform(0, 1, 1)
y = np.random.binomial(1, 0.5)

In [176]:
def analytic_grad(x) : 

    ### WRITE CODE BELOW ###
    z=np.dot(np.transpose(w), x)+b  #
    y_cap=1/(1+np.exp(-z))
    gradient=(y-y_cap)*np.transpose(w)
    return gradient
    ### WRITE CODE ABOVE ###

# *Problem 1(b)*

Now, we compute the numeric gradient for the function L. Refer to [this](https://stackoverflow.com/questions/38854363/is-there-any-standard-way-to-calculate-the-numerical-gradient) stack exchange post to see how numeric gradients are computed

In [177]:
def L(x):
    return y*(np.log(1/(1+np.exp(-np.dot(np.transpose(w), x)+b))))+(1-y)*(np.log(1-(1/(1+np.exp(-np.dot(np.transpose(w), x)+b)))))

def numeric_grad(x) : 

    ### WRITE CODE BELOW ###
    eps=1e-8
    h=(eps**0.5)*x
    gradient=(L(x+h)-L(x-h))/2*h
    
    return gradient
    ### WRITE CODE ABOVE ###

# *Problem 2*

Here, we'll be looking at computational graphs, and their calculus, finding gradients of  variables with respect to other variables in the graph.



We'll be looking at nodes of the graph that are operation based, i.e., each operation performed has a node associated with it.

We'll provide you with example implementations for three of the nodes, the *add* node, the *nth power* node and the *sine* node, you'll have to write the classes for all the other nodes.

# **Multi-input nodes**

In [178]:
class Add: 
  
  # A class to add multiple variables together
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # Scalar addition is taken care of in a separate node, we could have fit into this node but thought it might be easier if it wasn't.
                               # So cases like "a + 1" are to be done separately, and cases like "b + c + d + 4" can be done as "(b + c + d) + 4" or "b + c + (d + 4)", since our scalar addition supports only one variable and one scalar, we'll get to that later
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    return np.sum(self.lst_values) # This computes the sum of all the variables
  
  def compute_gradients(self):
    return dict(zip(self.lst_names[0], np.ones(len(self.lst_names[0])))) # This gives the gradient of the sum wrt to all the input variables, as a dictionary, will be used later


class Multiply: 
  
  #Everything's almost the same as the add class, but this deals with multiplication of more than 1 variables
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # Scalar multiplication is taken care of in a separate node
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    return np.prod(self.lst_values[0])
  
  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    gradients=[]
    for i in range(len(self.lst_names[0])):
        gradients.append(self.compute_output()/(self.lst_values[0][i]))
    return dict(zip(self.lst_names[0], gradients))



# **Scalar multiplication/addition nodes**

In [179]:
class Add_Scalar: 
  
  # A class to add a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be added, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be added.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be added, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    self.lst_names[1]=list(map(int, self.lst_names[1]))
    arr=np.array([self.lst_values[0], self.lst_names[1]])
    
    return np.sum(arr, axis=0).tolist()

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    return dict(zip(self.lst_names[0], np.ones(len(self.lst_values))))


class Multiply_Scalar: 
  
  # A class to multiply a variable with a scalar
  def __init__(self, lst_names, lst_values):
    self.lst_names = lst_names # This numpy arr contains all the variable names that are to be multiplied, with each variable name in datatype str.
                               # This list must only have 2 elements, the first should be the string corresponding to the name of the variable, and the second should be a string of the scalar value to be multiplied.
    self.lst_values = lst_values # This numpy arr contains all the variable values that are to be multiplied, as scalars.

  def compute_output(self):
    # Write your code to compute the output of this operation
    out=self.lst_values[0][0]*float(self.lst_names[1][0])
    return out

  def compute_gradients(self):
    # Write your code to create a dictionary, storing all the gradients of the output wrt to each input (In this case there is only a single input that matters)
    return {self.lst_names[0][0]: float(self.lst_names[1][0])}
    

# **Nodes for special functions**

In [180]:
class Power:

  def __init__(self, name, value, exponent):
    self.name = name # Name of the variable to be exponentiated
    self.value = value # Value of the variable
    self.exponent = exponent # Value of the exponent
  
  def compute_output(self):
    return np.power(self.value, self.exponent)
  
  def compute_gradients(self):
    return {self.name : self.exponent*(np.power(self.value, (self.exponent - 1)))}

class Sine: 

  # A class to apply the sine function on a variable
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    return np.sin(self.value)
  
  def compute_gradients(self):
    return {self.name : np.cos(self.value)}


class Logarithm:

  # A class to compute the logarithm of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.log(self.value)

  def compute_gradients(self):
    # Write your code here
    return {self.name: 1/(self.value)}

class Exponential: 

  # A class to compute the exponential of a value
  def __init__(self, name, value):
    self.name = name
    self.value = value
  
  def compute_output(self):
    # Write your code here
    return np.exp(self.value)

  def compute_gradients(self):
    # Write your code here
    return {self.name: np.exp(self.value)} 



Now that we have these classes, let's use them to actually find gradients of complex networks. We finally bring in the idea of a computational graph, which makes it much easier for the gradients to be computed.

This is the image of the example graph that we want you to work with. 





<div>
<img src="https://drive.google.com/uc?id=1VtI1lf85bG8cO1u_8l0D0rqVGhwHQtPK"
 width="500" height="500">
</div>

\begin{equation}
d = 3a \\ 
e = abc \\ 
f = sin(c) \\ 
g = exp(e) \\ 
h = a + d + g + f
\end{equation}

# **Forward Propogation**

In [181]:
def forward_prop(a, b, c) : 
    '''
    Input : real numbers a, b, c.

    Outputs : A dictionary of the values at each node with keys as the names of nodes
    Grads : A dictionary of the gradients at each edge with keys as a pair of nodes 
    
    e.g. Grads["ce"] = ...
    '''
    Outputs={'a': a, 'b': b, 'c': c}
    Grads={'aa': 1, 'bb': 1, 'cc': 1}
    multiply_scalar=Multiply_Scalar([["d"],["3"]], [[a]])
    Outputs['d']=multiply_scalar.compute_output()
    Grads={**Grads, **{'da': multiply_scalar.compute_gradients()['d']}}
    multiply=Multiply([['a', 'b', 'c']], [[a, b, c]])
    Outputs['e']=multiply.compute_output()
    x=multiply.compute_gradients()
    x['ea']=x.pop('a')
    x['eb']=x.pop('b')
    x['ec']=x.pop('c')
    Grads={**Grads, **x}
    del x
    sine=Sine('f', c)
    Outputs['f']=sine.compute_output()
    Grads={**Grads, **sine.compute_gradients()}
    Grads['fc']=Grads.pop('f')
    exp=Exponential('g', Outputs['e'])
    Outputs['g']=exp.compute_output()
    Grads={**Grads, **exp.compute_gradients()}
    Grads['ge']=Grads.pop('g')
    add=Add([["a", "d", "g", "f"]], [[a, Outputs['d'], Outputs['g'], Outputs['f']]])
    Outputs["h"]=add.compute_output()
    d=add.compute_gradients()
    d['ha']=d.pop('a')
    d['hd']=d.pop('d')
    d['hg']=d.pop('g')
    d['hf']=d.pop('f')
    Grads={**Grads, **d}
    return (Outputs, Grads)

# **Backward Propogation**

Most of the usage of computational graphs in Machine Learning centers around finding the gradients of a particular loss, wrt to all the parameters. Here, your task is to do a simpler version of that.

Use the classes you wrote to calculate the following gradients : 


*   $ \frac{\partial h}{\partial a}$
*   $ \frac{\partial h}{\partial b}$
*   $ \frac{\partial h}{\partial c}$

Use the technique of *backpropogation* to code out the gradients step-wise, along all possible chains of the graph starting from $h$ and ending at $a,b,c$ respectively. 

Don't try to directly get these values without backpropogation, it might be easier here, but with complicated neural networks it wouldn't be :) 



In [182]:
def all_paths(adj_list, start, end, index):
    def dfs(start, index, path, output):
        if start==end:
            output.append(path)
        for i in adj_list[start]:
            dfs(i, index, path+[i], output)
    output=[]
    dfs('h', index, ['h'], output)
    return output
    

def backward_prop(a, b, c, Outputs, Grads) : 
    '''
    Input : a, b, c (input layer); Outputs (values at each node); Grads (gradients stored at each edge)
    Retuns : result (gradients w.r.t a, b, c)
    '''
    h_a=[]
    h_b=[]
    h_c=[]
    result=[]
    grad_keys=list(Grads.keys())
    
#Creating Adjacency List

    adj_list={'a': [], 'b': [], 'c': []}
    v=['d', 'e', 'f', 'g', 'h']
    for i in v:
        l=[]
        for j in grad_keys:
            if j[0]==i:
                l.append(j[1])
        adj_list={**adj_list, **{i: l}}

    index={'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
    h_a=all_paths(adj_list, 'h', 'a', index)
    h_b=all_paths(adj_list, 'h', 'b', index)
    h_c=all_paths(adj_list, 'h', 'c', index)
    
#Restructuring h_a, h_b and h_c and appending gradients to result

    for i in h_a:
        for j in range(len(i)-1):
            i[j]=i[j]+i[j+1]
        del i[j+1]
    s=0
    for i in h_a:
        if(len(i)!=1):
            t=1
            for j in i:
                t=t*Grads[j]
        else:
            t=Grads[i[0]]
        s=s+t
    result.append(s)
    
    for i in h_b:
        for j in range(len(i)-1):
            i[j]=i[j]+i[j+1]
        del i[j+1]
    s=0
    for i in h_b:
        if(len(i)!=1):
            t=1
            for j in i:
                t=t*Grads[j]
        else:
            t=Grads[i[0]]
        s=s+t
    result.append(s)
    
    for i in h_c:
        for j in range(len(i)-1):
            i[j]=i[j]+i[j+1]
        del i[j+1]
    s=0
    for i in h_c:
        if(len(i)!=1):
            t=1
            for j in i:
                t=t*Grads[j]
        else:
            t=Grads[i[0]]
        s=s+t
    result.append(s)
    
    return result

# **Combining both processes**

For the purpose of values, assume that $a = 3, b = 1, c = 2$. Here, we call both forward and backward propogation functions to demonstrate functionality of the functions above.

In [183]:
# Initialisation
a = 3
b = 1
c = 2

# Forward propogation
(Outputs, Grads) = forward_prop(a, b, c)
print(f"Value obtained upon forward propogation : {Outputs['h']}")

# Backward propogation
result = backward_prop(a, b, c, Outputs, Grads)
print(f'Values of gradients are : {result[0]}, {result[1]}, {result[2]}')

Value obtained upon forward propogation : 416.3380909195608
Values of gradients are : 810.8575869854702, 2420.5727609564105, 1209.8702336416582


# **Submission Instructions**

Upload this notebook on your github classroom repository by the name Week1.ipynb